In [ ]:
import sys
import pathlib as pl
import math

import numpy as np
import geopandas as gpd
import shapely.geometry as sg
import matplotlib.pyplot as plt
from loguru import logger

sys.path.append("../")

import geohexgrid as ghg


%load_ext autoreload
%autoreload 2

DATA_DIR = pl.Path("../tests/data")
NZTM = "epsg:2193"  # New Zealand Transverse Mercator CRS
WGS84 = "epsg:4326"

In [ ]:
nrows, ncols, R = 1000, 1000, 1 
a0, b0 = -2, -2
x0, y0 = ghg.double_to_cartesian(a0, b0, R)
%time grid = ghg.make_grid(nrows, ncols, R, x0, y0, a0=a0, b0=b0)


In [ ]:
# make_grid_points and make_grid

nrows, ncols, R = 3, 4, 1 
a0, b0 = -2, -2
x0, y0 = ghg.double_to_cartesian(a0, b0, R)
X, Y = ghg.make_grid_points(nrows, ncols, R, x0, y0)
grid = ghg.make_grid(nrows, ncols, R, x0, y0, a0=a0, b0=b0)
display(grid.head())
print(grid["cell_id"].tolist())

fig, ax = plt.subplots()
ax.scatter(X, Y, color="black", alpha=0.9)
grid.plot(ax=ax, color="white", edgecolor="red", alpha=0.9, aspect="equal")
# Plot labelling from https://stackoverflow.com/a/38902492
grid.apply(
    lambda x: ax.annotate(
        text=x['cell_id'], 
        xy=x.geometry.centroid.coords[0], 
        ha='center', 
        color="red"
    ), 
    axis=1
);

# Areas should be correct
assert np.allclose(grid.area, 3 * np.sqrt(3) * R**2 / 2)

In [ ]:
grid = ghg.make_grid(2, 3, R=1, x0=2, y0=1)
display(grid)
grid["cell_id"].tolist()

p = grid.union_all().boundary
display(p, p.is_ring)

p1 = grid["geometry"].iat[0].buffer(-0.001)
p2 = grid["geometry"].iat[1]
q = p1.union(p2).boundary

display(q, q.is_ring)


In [ ]:
# make_grid_from_bounds

R = 1
r = ghg.K * R
rect = gpd.GeoDataFrame(geometry=[sg.Polygon([(-0.6*R, -0.1), (2.2*R, -0.1), (2.2*R, 3.2*r), (-0.6*R, 3.2*r)])])
grid = ghg.make_grid_from_bounds(*rect.total_bounds, R=R) #, ox=-2*R/3, oy=-2*R/3)
display(grid.head())
print(grid["cell_id"].tolist())

base = rect.plot(color='gray', aspect="equal")
grid.plot(ax=base, color="white", edgecolor="red", alpha=0.5)

In [ ]:
# make_grid_from_bounds

R = 1
r = ghg.K * R
rect = gpd.GeoDataFrame(geometry=[sg.Polygon([(-R/2, -r), (R/2, -r), (R/2, r), (-R/2, r)])])
grid = ghg.make_grid_from_bounds(*rect.total_bounds, R=R, ox=-3*R/2, oy=-r)
display(grid.head())
print(grid["cell_id"].tolist())

base = rect.plot(color='gray', aspect="equal")
grid.plot(ax=base, color="white", edgecolor="red", alpha=0.5)

In [ ]:
rect = gpd.GeoDataFrame(geometry=[sg.Polygon([(0, 0), (1, 0), (1, 1), (0, 1)])])
for grid in [
    ghg.make_grid_from_bounds(*rect.total_bounds, R=0.27, ox=0, oy=0),
    ghg.make_grid_from_bounds(*rect.total_bounds, R=0.2, ox=0, oy=0),
    ghg.make_grid_from_bounds(*rect.total_bounds, R=0.5, ox=0, oy=0.1),
    ghg.make_grid_from_bounds(*rect.total_bounds, R=1, ox=0.6, oy=0),
]:
    fig, ax = plt.subplots()
    rect.plot(ax=ax, color="red")
    grid.plot(ax=ax, color="blue", alpha=0.5)
    plt.show()


In [ ]:
g = gpd.GeoDataFrame({"geometry": [sg.LineString([(0, 0), (1, 0), (1, 1), (0, 1)])]})
for i in range(0, 20):
    R = (i + 1)/10 
    r = ghg.K*R
    grid = ghg.make_grid_from_gdf(g, R=R, ox=-3*R/2 - 0.1, oy=-r - 0.1)
    assert grid.union_all().contains(g.union_all())

    fig, ax = plt.subplots()
    g.plot(ax=ax, color="red")
    grid.plot(ax=ax, color="blue", alpha=0.5)
    plt.show()

In [ ]:
shape = gpd.GeoDataFrame(geometry=[sg.Polygon([(1, -1), (3, 1), (0, 3)])])
grid = ghg.make_grid_from_gdf(shape, R=1)

display(grid.head())
print(grid["cell_id"].tolist())

base = shape.plot(color='gray', aspect="equal")
grid.plot(ax=base, color="white", edgecolor="red", alpha=0.5)

In [ ]:
# make_grid_from_gdf

shapes = gpd.read_file(DATA_DIR / "shapes.geojson").to_crs(NZTM)
R = 900
grid = ghg.make_grid_from_gdf(shapes, R=R)
display(grid.head())

assert grid.union_all().contains(shapes.union_all())

base = shapes.plot(color='gray', aspect="equal")
grid.plot(ax=base, color="white", edgecolor="red", alpha=0.5)


In [ ]:
# Prettier test

shapes = gpd.read_file(DATA_DIR / "nz_tas.gpkg")
display(nz.crs)
%time grid = ghg.make_grid_from_gdf(shapes, R=10_000) # 10 km circumradius

assert grid.union_all().contains(shapes.union_all())

base = shapes.plot(color='black', aspect="equal", figsize=(10, 10))
grid.plot(ax=base, color='white', edgecolor="red", alpha=0.5)

plt.axis('off')
plt.savefig('../nz_10000m.png', bbox_inches='tight')

In [ ]:
# Speed test. Takes about 15 seconds on my computer.

nz = gpd.read_file(DATA_DIR / "nz_tas.gpkg")
akl = nz.loc[lambda x: x["ta2021_name"] == "Auckland"]
%time grid = ghg.make_grid_from_gdf(akl, R=250) 

base = akl.plot(color='black', figsize=(20, 20), aspect="equal")
grid.plot(ax=base, color='white', edgecolor="red", alpha=0.5)

assert grid.union_all().contains(akl.union_all())

